# ABOUT
This notebook is meant for translating normal A4 PDF to A3 booklets optimised for printing at TU Delft
1. saves money
2. saves paper
3. saves electricity, by printing half the number of pages
4. dont have to deal with annoying flips in single stapled documents


# Methodology
1. Make booklet
    1. first arrange the PDF and insert the blank pages at the right places: the code hence formed returns a PDF with single pages in the reverse order
    2. after single page booklet-order formation the booklet needs to be formed in A3
2. Rotate pages
    * once the A3 booklet like Adobe booklet is formed, we rotate every even numbered page by 90 degrees and odd ones by -90 degrees so as to make it printable in A3 at TU Delft printers, since the printers only print along the short edge of A3

# Booklet formation

This [Link for theory](https://wiki.scribus.net/canvas/Making_a_booklet_with_Scribus_and_Adobe_Reader) explains that the process for booklet formation is known as imposition

There are SDK's available for imposing PDFs online, however the following code block does the same procedure
<!-- [another link](https://stackoverflow.com/questions/45226318/pdf-imposition-using-python) -->

we thought that we will be using the library pdfimpose from pip for this purpose, found [here](https://pypi.org/project/pdfimpose/), however the docs are quite confusing, and it was easier to implement the required imposition logic ourselves.

what we want to do is found in the [docs of the library](https://pdfimpose.readthedocs.io/en/latest/library/#module-pdfimpose.schema.saddle)



The following cell has code adapted from a fellow stackoverflow user for the purpose of imposing a given PDF as a booklet, as it would have been the case in Adobe Acrobat Booklet formation

In [16]:
import os
import math
import numpy as np
from PyPDF2 import PdfFileWriter as Writer, PdfFileReader as Reader

def main_impose(pdf_path):
    """
    Summary: This function takes the PDF (A4 sized) as input and returns an 
             A3 size PDF file which has booklet with every even numbered page 
             rotated by 90 degrees and odd by -90 degrees so that the PDF file
             is ready for sending to the printportal at TU Delft. 
             The output booklet is in a new folder named tempBookletOutput.
    ___________________
    Params:
        pdf_path (str): path of pdf to be formatted for printing 
    ___________________
    Returns:    None
    """
    # define A4 sizes in portrait
    A4_width, A4_height = 210, 297
    # define A3 sizes in portrait
    A3_width, A3_height = 841.8897637795, 1190.5511811024

    # read PDF
    orig = Reader(open(pdf_path, 'rb'))
    origpages = orig.pages
    orig_height, orig_width = orig.pages[0].mediaBox[2], orig.pages[0].mediaBox[3]

    # initialise writer for new PDF
    new = Writer()
    
    nop = len(origpages)
    nop_booklet = int(math.ceil(nop / 4.0))

    base = np.arange(nop_booklet) + 1
    base = 2 * base
    base = base[::-1] #reverses the list
    pages = [] # list to arrange the page numbers 
    
    for i in base:
        num = nop_booklet * 4 + 1
        pages.append(i)
        pages.append(num - i)
        pages.append(num - i + 1)
        pages.append(num - (num - i + 1))
    
    bookletOrderedPagesTuples = list((pages[i],pages[i+1]) for i in range(0,len(pages),2) )[::-1]
    bookletOrderedPagesList = [item for sublist in bookletOrderedPagesTuples for item in sublist]
    
    # boolean to define left right
    left_bool = True
    # adding pages to the new PDF
    for i in bookletOrderedPagesList:
        if left_bool==True:
            newPages = new.addBlankPage(width = A3_height, height = A3_width)

        if i > nop:
            # do nothing
            # new.addBlankPage(width=orig_height, height=orig_width)
            left_bool = not left_bool

        else:
            idx = int(i - 1)
            # new.addPage(origpages[idx])
            
            if left_bool:
                newPages.mergePage( origpages[idx] )
            else:
                # newPages.mergeRotatedScaledTranslatedPage(origpages[idx] , 0, 0.25, a3_height/2, 0, expand=False)
                newPages.mergeTranslatedPage(origpages[idx], A3_height/2, 0, expand=False)

            # swap positions left to right by changing the boolean value
            left_bool = not left_bool
    
    # make folder if ont exists
    outpath = os.path.join(os.path.dirname(pdf_path), "tempBookletOutput")
    if not os.path.exists(outpath) :
        os.mkdir(outpath)

    # save the modified pdf file
    bookletName = os.path.basename(pdf_path)[:-4] + "_booklet.pdf"
    
    fn = os.path.join(outpath, bookletName)

    for page in range( int(new.getNumPages()) ): 
        # creating rotated page object 
        pageObj = new.getPage(page)
        # rotate if even page, page is odd
        if page%2 != 0:
            # print("here")
            pageObj.rotateClockwise(90) 
        else:
            pageObj.rotateClockwise(-90) 

    with open(fn, 'wb') as f:
        new.write(f)
        
    print("File saved as {}".format(fn))

# Folder Run
Run the following cell to input the folder where your A4 sized PDF files are placed which need to be printed at TU Delft.

You will end up with a new folder named as `tempBookletOutput` with the requisite modified PDF files.

In [17]:
# path = path to the folder containing the pdf files
path = input("Enter the path to folder containing the PDF: ")

j = [x for x in os.listdir(path) if x.endswith(".pdf")]
pathlist = [ os.path.join(path, x) for x in j ] # list of paths of pdf in folder

# impose and rotate pages
for filepath in pathlist:
    main_impose(filepath)


6
File saved as ./tempBookletOutput/saddle_09_booklet.pdf
6
File saved as ./tempBookletOutput/saddle_10_booklet.pdf
6
File saved as ./tempBookletOutput/saddle_11_booklet.pdf
6
File saved as ./tempBookletOutput/saddle_12_booklet.pdf


# Future Work
check for scaling of pages and ensure that no matter what the input page size is, it doesnt affect the output, right now it assumes that you are providing only A4 size pages